In [1]:
!pip install gradio langchain langchain_community langchain_milvus transformers pymilvus


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.3/248.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 650.6 kB/s eta 0:00:00


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [4]:
import os
import gradio as gr
import tempfile
from langchain_milvus import Milvus
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# Replicate token and model setup
os.environ['REPLICATE_API_TOKEN'] = "r8_3HHrig1gtQF2CZjxSVBiyGXIHrx33Yc2ee1BI"
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(model=model_path, replicate_api_token=os.environ['REPLICATE_API_TOKEN'])

# 📦 Temporary Milvus DB
db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

# 📄 Financial Literacy Content
filename = "finance_literacy.txt"
with open(filename, "w") as f:
    f.write("""
UPI: Unified Payments Interface (UPI) is a real-time payment system developed by NPCI. It allows users to link bank accounts and make instant money transfers via mobile.

Avoiding Online Scams: Never share your OTP, UPI PIN, or passwords. Beware of links promising free rewards. Use official banking apps only.

Interest Rates: Interest is the cost of borrowing money. Loans usually have an interest rate. Compare rates from different banks before applying.

Budgeting: Create a monthly budget by listing your income and expenses. Prioritize needs over wants. Track your spending to save money.

Digital Wallets: Apps like PhonePe, Google Pay, Paytm allow you to pay bills, recharge, and send money securely.

Safe Loan Practices: Always read terms before taking a loan. Choose loans with lower interest and no hidden fees.

How to use BHIM app: Download, register with your mobile number linked to your bank, set UPI PIN, and start sending or requesting money.

Fraud Protection: Use two-factor authentication. Don’t save card details on unknown websites. Report suspicious activity to your bank immediately.Explain about How to use Phone pe. Explain Google pay also
""")

# Load & Split Content
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=tokenizer.model_max_length // 2,
    chunk_overlap=0,
)
texts = splitter.split_documents(documents)
for i, doc in enumerate(texts):
    doc.metadata["doc_id"] = i + 1
vector_db.add_documents(texts)

#  Financial Literacy Prompt
template = """
You are a helpful Digital Financial Literacy Assistant.
User Question: {question}
Give a simple, culturally-inclusive explanation in easy language.
Include tips to stay safe, explain tools like UPI, wallets, loans, interest, scams, budgeting.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Chains
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

#  Gradio Interface
def ask_finance_agent(query):
    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"

iface = gr.Interface(
    fn=ask_finance_agent,
    inputs=gr.Textbox(label="Ask a question about digital finance", placeholder="e.g. How to use UPI?"),
    outputs=gr.Textbox(label="Answer"),
    title="AI Agent for Digital Financial Literacy",
    description="Ask about UPI, loans, scams, budgeting, and digital tools. This assistant helps you stay safe and informed.",
    theme="default"
)

iface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/tmp/ipython-input-3987640662.py:75: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=prompt)
/tmp/ipython-input-3987640662.py:76: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_chain = StuffDocumentsChain(llm_chain=llm_chain)
/tmp/ipython-input-3987640662.py:78: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://pyth

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c10a38c6115e602088.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
